In [1]:
from huggingface_hub import hf_hub_download
import torch

from open_flamingo import create_model_and_transforms

/home/liuxiao/anaconda3/envs/flamingo/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import os

In [22]:
# model, image_processor, tokenizer = create_model_and_transforms(
#     clip_vision_encoder_path="ViT-L-14",
#     clip_vision_encoder_pretrained="openai",
#     lang_encoder_path="anas-awadalla/mpt-7b",
#     tokenizer_path="anas-awadalla/mpt-7b",
#     cross_attn_every_n_layers=4# cache_dir="PATH/TO/CACHE/DIR"  # Defaults to ~/.cache
# )

# checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-9B-vitl-mpt7b", "checkpoint.pt")
# model.load_state_dict(torch.load(checkpoint_path), strict=False)
model, image_processor, tokenizer = create_model_and_transforms(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path="togethercomputer/RedPajama-INCITE-Instruct-3B-v1",
    tokenizer_path="togethercomputer/RedPajama-INCITE-Instruct-3B-v1",
    cross_attn_every_n_layers=2
)


checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-4B-vitl-rpj3b-langinstruct", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50280. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Flamingo model initialized with 1089480736 trainable parameters


checkpoint.pt:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

_IncompatibleKeys(missing_keys=['vision_encoder.class_embedding', 'vision_encoder.positional_embedding', 'vision_encoder.proj', 'vision_encoder.conv1.weight', 'vision_encoder.ln_pre.weight', 'vision_encoder.ln_pre.bias', 'vision_encoder.transformer.resblocks.0.ln_1.weight', 'vision_encoder.transformer.resblocks.0.ln_1.bias', 'vision_encoder.transformer.resblocks.0.attn.in_proj_weight', 'vision_encoder.transformer.resblocks.0.attn.in_proj_bias', 'vision_encoder.transformer.resblocks.0.attn.out_proj.weight', 'vision_encoder.transformer.resblocks.0.attn.out_proj.bias', 'vision_encoder.transformer.resblocks.0.ln_2.weight', 'vision_encoder.transformer.resblocks.0.ln_2.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.bias', 'vision_encoder.transformer.resblocks.1.ln_1.weight', 'vision_encoder.transformer.resbloc

In [27]:
model.to(1)

Flamingo(
  (vision_encoder): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1024,), eps=1e-0

In [26]:
from PIL import Image
import requests
import torch

"""
Step 1: Load images
"""
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)

demo_image_two = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
        stream=True
    ).raw
)

query_image = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028352.jpg", 
        stream=True
    ).raw
)


"""
Step 2: Preprocessing images
Details: For OpenFlamingo, we expect the image to be a torch tensor of shape 
 batch_size x num_media x num_frames x channels x height x width. 
 In this case batch_size = 1, num_media = 3, num_frames = 1,
 channels = 3, height = 224, width = 224.
"""
vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
vision_x = torch.cat(vision_x, dim=0)
vision_x = vision_x.unsqueeze(1).unsqueeze(0)#.to(0)

"""
Step 3: Preprocessing text
Details: In the text we expect an <image> special token to indicate where an image is.
 We also expect an <|endofchunk|> special token to indicate the end of the text 
 portion associated with an image.
"""
tokenizer.padding_side = "left" # For generation padding tokens should be on the left
lang_x = tokenizer(
    ["<image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of"],
    return_tensors="pt",
)#.to(0)


"""
Step 4: Generate text
"""
generated_text = model.generate(
    vision_x=vision_x,
    lang_x=lang_x["input_ids"],
    attention_mask=lang_x["attention_mask"],
    max_new_tokens=20,
    num_beams=3,
).cpu()

print("Generated text: ", tokenizer.decode(generated_text[0]))

Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


Generated text:  <image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of a buffet table.<|endofchunk|>


In [10]:
vision_x.shape

torch.Size([1, 3, 1, 3, 224, 224])

In [31]:
img_root = "/home/liuxiao/TuringGithub/imagecaption/MturkExps/image_captioning_task/Data/imgset"
answers = {}
captioning_prompt = '''Question: What are things in this picture? Describe the scene with at least 6 words. Answer:'''
tokenizer.padding_side = "left"
for i, fn in enumerate([1,2,3,4,5]): #(os.listdir(img_root)):
    img_path = os.path.join(img_root, '000{}.jpg'.format(fn))

    query_image = Image.open(img_path)
    vision_x = image_processor(query_image).unsqueeze(0)
    vision_x = vision_x.unsqueeze(1).unsqueeze(0).to(1)
    lang_x = tokenizer(
    ["<image>{}".format(captioning_prompt)],
    return_tensors="pt",
    ).to(1)
    generated_text = model.generate(
        vision_x=vision_x,
        lang_x=lang_x["input_ids"],
        attention_mask=lang_x["attention_mask"],
        max_new_tokens=50,
        num_beams=3,
        pad_token_id=tokenizer.eos_token_id
    ).cpu()    # cleaned_response = [re.sub(r'[^a-zA-Z]', '', _).lower() for _ in answer_text.split(',')]
    # answers.setdefault(fn,[]).extend(cleaned_response)
    answers[fn] = tokenizer.decode(generated_text[0]) #.replace("This is a photo of")
#     break

In [32]:
answers

{1: '<image>Question: What are things in this picture? Describe the scene with at least 6 words. Answer: Carrots, broccoli, and a potato.<|endofchunk|>',
 2: '<image>Question: What are things in this picture? Describe the scene with at least 6 words. Answer: A man is selling watermelons in a market.<|endofchunk|>',
 3: '<image>Question: What are things in this picture? Describe the scene with at least 6 words. Answer: A girl is hugging a computer.<|endofchunk|>',
 4: '<image>Question: What are things in this picture? Describe the scene with at least 6 words. Answer: A man with a pitchfork.<|endofchunk|>',
 5: '<image>Question: What are things in this picture? Describe the scene with at least 6 words. Answer: A man in a red ski suit is standing on a snowy slope.<|endofchunk|>'}